## Lab3 Molecular Dynamics

In this lab, we will prepare and run the necessary steps to perform MD simulations of the cannabinoid 2 receptor. We use the crystal structure 6kpf (https://www.rcsb.org/structure/6KPF) as a starting point.

The lab includes a series of steps to interact with the GROMACS program, as well as with a few python script included in the python package PyMemDyn. Normally, these steps are all run at once, but here we try to run them individually to better understand which steps are needed to succesfully run MD on a membrane protein.

I reccomend to read up on pymemdyn and related webserver here:
https://portlandpress.com/biochemsoctrans/article-abstract/41/1/205/68208/Characterization-of-the-dynamic-events-of-GPCRs-by
and
https://academic.oup.com/nar/article/44/W1/W455/2499371?login=true

We have pregenerated the ligand parameters using LigParGen:
http://zarbi.chem.yale.edu/ligpargen/

Below, you will find the relevant code blocks that you can execute by using the play button on top, or by typing shft+enter.

If you want to know more about what the code is doing, please ask one of the teachers!

In [1]:
#!/usr/bin/env python3.7
# Import python native functions
import argparse
import os
import shutil
import sys

# Add PyMemDyn modules
sys.path.insert(0, '../')

import complex
import gromacs
import membrane
import protein
import pymemqueue
import settings
import recipes
from pymemdyn import Run

In [2]:
!cp data/clean/* rundir/

In [3]:
# First, we instantiate the run class of pymemdyn, this top level object takes run time variables
# and stores them in an object so we can use them in the next steps
os.chdir('rundir')
run = Run(own_dir = os.getcwd(),
          repo_dir = settings.TEMPLATES_DIR,
          pdb = 'protein.pdb',
          ligand = 'E3R',
          alosteric = None,
          waters = None,
          ions = None,
          cho = None,
          ligpargen = 'l',
          library = None,
          restraint = 'bw',
          queue = "",
          debug = True
)
# Clean up old files
run.clean()

True

This command returns a list of each of the steps run by the program pymemdyn.
We will use these keywords in the next command block to run the individual steps
in the program.
Note that pymemdyn can also run everything in one go (this was the intention of the 
program), but for teaching purposes it makes more sense to carefully look at each individual step.

In [4]:
run.moldyn_notebook_info()

pdb2gmx
set_itp
concat
editconf
set_protein_size
editconf2
set_protein_size2
set_popc
editconf3
make_ndx_lig
genrestr_lig
editconf4
make_topol
editconf5
solvate
set_water
editconf6
editconf7
solvate2
count_lipids
make_topol2
make_topol_lipids
make_ffoplsaanb
set_grompp
set_chains
make_ndx
grompp
trjconv
get_charge
genion
grompp2
trjconv2
grompp3
trjconv3
clean_pdb


You can use the code block below to run each of the commands individually, there will be a short description of each of the
steps printed out. The first step (pdb2gmx) has already been filled in so you know what input to use. Some commands are running via GROMACS (indicated in the header) other commands are actual python
scripts part of the pymemdyn package.

Sometimes the help function mentions a filetype (.top or .itp for instance) if you don't know what these files are
you can find more information here:
https://manual.gromacs.org/documentation/5.1.1/user-guide/file-formats.html

In [30]:
run.moldyn_notebook_run("set_grompp")

LigandInit : set_grompp.
Running the following command
{'command': 'set_grompp', 'options': {'steep.mdp': 'steepDEBUG.mdp', 'popc.itp': 'popc.itp', 'spc.itp': 'spc.itp', 'ions.itp': 'ions.itp', 'ffoplsaabon_mod.itp': 'ffoplsaabon_mod.itp', 'ffoplsaa_mod.itp': 'ffoplsaa_mod.itp'}}

========================INFO============================

TEST

========================INFO END========================



FileNotFoundError: [Errno 2] No such file or directory: '/home/willem/pymemdyn-teaching/templates/steepDEBUG.mdp'

In [8]:
# Ater you have completed all steps above, you would normally start running the MD simulation. However it takes a few hours to complete,
# so we have already done it for you.
# The output from a PyMemDyn run can be found in the finalOutput folder, which we copy in here now

!cp -r ../data/finalOutput .

# Note, it takes a bit of time before all files are transferred.

In [9]:
!pwd

/home/willem/pymemdyn-teaching/notebook/rundir
